# 文本大模型与多模态大模型介绍

## 文本大模型

文本大模型，例如step-1系列，基于生成式人工智能技术，通过在大量数据上进行训练，以理解和生成自然语言。

## 文本大模型可以用于但不限于以下任务：

- 内容生成：生成文章、故事、对话等。
- 代码生成：根据描述生成代码片段。
- 摘要：将长篇文章或文档缩减为简短的摘要。
- 翻译：将一种语言翻译成另一种语言。
- 问答：回答各种问题，从简单的事实查询到复杂的推理问题。
- 创意写作：创作诗歌、歌词或其他形式的创意文本。
- 语言理解：进行情感分析、文本分类等。
- 对话系统：与用户进行自然语言对话，模拟聊天机器人或虚拟助手。

使用这些模型时，通常需要提供一个prompt，即一段引导性的文本，它告诉模型需要执行的任务类型以及相关的上下文信息。设计一个有效的prompt是至关重要的，因为它直接影响到模型输出的质量和相关性。

## 多模态大模型

多模态大模型，在文本大模型基础上，增加了对多种模态（如图像、音频、视频等）数据的处理能力，如step-1v系列，目前能够理解处理文本以及图像。多模态大模型的核心优势在于更符合人类认知世界的习惯，因为人类通过多种感官接收多模态信息，这些信息通常是互补和协同的，使用多模态大模型可以更好地理解这些信息以及进行推理。

## 多模态大模型可以用于但不限于以下任务：

- 图像到文本的翻译：为图像生成描述性文本。
- 对话系统：结合文本和图像信息回答复杂问题。
- 创作：根据图像和文本创作文学作品。
- 医疗影像诊断：结合医学影像（如X光片、MRI图像）和病人的病历文本，多模态模型可以帮助医生更准确地诊断疾病。
- 搜索引擎：搜索引擎可以利用多模态技术来理解用户的查询意图，结合文本、图像，提供更相关的搜索结果。
- 自动驾驶：自动驾驶汽车可以使用多模态模型来处理来自多个传感器（如摄像头、雷达、激光雷达）的数据，以更好地理解周围环境并做出驾驶决策。

# 文档解析API使用教程

本教程将介绍如何使用阶跃星辰(StepFun)的文档解析API，通过一系列示例代码展示如何上传、管理和处理文档文件。

## 1. 文件上传

首先，我们需要上传一个文件到API服务器。以下代码展示了如何上传文件并获取文件ID。

In [ ]:
import requests
import os

STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
BASE_URL= os.environ['STEPFUN_ENDPOINT']

headers = {
    "Authorization": f"Bearer {STEP_API_KEY}"
}
files = {
    "file": ("lihua.jpg", open("../Image/lihua.jpg", "rb")),
    #"file": ("测试音频.mp3", open("../audio/output/测试音频.mp3", "rb")),
    # "url": (None, "https://arxiv.org/pdf/2402.01684"),
}
data = {
    # "purpose": "storage"
    # "purpose": "file-extract"
    "purpose": "retrieval-image"
}

response = requests.post(BASE_URL+"/files", headers=headers, files=files, data=data)

print(response.json())

## 2. 获取文件状态

上传文件后，我们可以检查文件的处理状态。以下代码展示了如何获取文件状态。

In [ ]:
import requests
import os

STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
BASE_URL= os.environ['STEPFUN_ENDPOINT']

file_id = "file-KIc1YSExFY"
URL = BASE_URL+"/files/"+file_id

headers = {
    "Authorization": f"Bearer {STEP_API_KEY}"
}

response = requests.get(URL, headers=headers)
responseJson = response.json()
print(response.json())
print(responseJson['status'])

## 3. 获取文件内容

一旦文件处理完成，我们可以获取文件的内容。以下代码展示了如何获取文件内容。

In [ ]:
import requests
import os

STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
BASE_URL= os.environ['STEPFUN_ENDPOINT']

file_id = "file-KIc1YSExFY"
URL = BASE_URL+"/files/"+file_id+"/content"

headers = {
    "Authorization": f"Bearer {STEP_API_KEY}"
}

response = requests.get(URL, headers=headers)

print(response.text)
print("返回码：", response.status_code)

## 4. 文件处理与模型调用

以下代码展示了一个完整的工作流程：上传文件、等待处理完成、获取内容，然后使用模型进行分析。

In [ ]:
import requests
import time
from openai import OpenAI
import os

STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
BASE_URL= os.environ['STEPFUN_ENDPOINT']

def upload_file(filename, filepath, base_url, step_api_key):
    headers = {
        "Authorization": f"Bearer {step_api_key}"
    }
    with open(filepath, "rb") as file:
        files = {
            "file": (filename, file),
        }
        data = {
            "purpose": "file-extract"
        }
        # Make the API request
        response = requests.post(f"{base_url}/files", headers=headers, files=files, data=data)
        # Parse the JSON response
        response_json = response.json()
        # Return the file ID
        return response_json.get("id")
def get_file_status(fileid, base_url, step_api_key):
    headers = {
        "Authorization": f"Bearer {step_api_key}"
    }
    url = f"{base_url}/files/{fileid}"
    response = requests.get(url, headers=headers)
    response_json = response.json()
    return response_json.get('status', 'unknown')

def get_file_content(fileid, base_url, step_api_key):
    headers = {
        "Authorization": f"Bearer {step_api_key}"
    }
    url = f"{base_url}/files/{fileid}/content"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        return f"Error: {response.status_code} - {response.text}"


# 上传文件获取文件ID
file_id = upload_file(
    filename="模型介绍.docx",
    filepath="./模型介绍.docx",
    base_url=BASE_URL,
    step_api_key=STEP_API_KEY,
)

# 等待文件解析完成
while True:
    status = get_file_status(file_id, BASE_URL, STEP_API_KEY)
    if status == 'success':
        break
    time.sleep(1)  # 避免频繁请求

# 获取文件内容
file_content = get_file_content(file_id, BASE_URL, STEP_API_KEY)
print(file_content)

# 生成对话
messages = [
    {
        "role": "system",
        "content": "你是阶跃星辰助手，会读取用户发送给你的文件的内容，并结合文件内容回答问题",
    },
    {
        "role": "user",
        "content": file_content,
    },
    {"role": "user", "content": "请简单介绍文档的具体内容"},
]
time_start = time.time()
# 初始化
client = OpenAI(api_key=STEP_API_KEY,base_url=BASE_URL)
# 针对文件大小，可以使用  step-1-256k 等超长上下文大模型
# 实现文本补全
response = client.chat.completions.create(
    messages=messages,
    model="step-1-256k",
    stream=True,
)
i=0
for chunk in response:
    i += 1
    print(chunk.choices[0].delta.content.strip(),end="")
    if i == 1:
        time_firstWord = time.time()
        elapsed_time= time_firstWord-time_start
        print(f"首字生成时间: {elapsed_time:.2f} 秒")
time_end = time.time()
elapsed_time = time_end - time_start
print(f"总生成时间: {elapsed_time:.2f} 秒")

## 5. 获取文件列表

以下代码展示了如何获取已上传文件的列表。

In [ ]:
import requests
import os

STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
BASE_URL= os.environ['STEPFUN_ENDPOINT']

#filePurpose根据需要填写file-extract或retrieval
filePurpose="file-extract"
URL = BASE_URL+"/files?purpose="+filePurpose

headers = {
    "Authorization": f"Bearer {STEP_API_KEY}"
}

response = requests.get(URL, headers=headers)
responseJson = response.json()
print(responseJson)

## 6. 删除文件

最后，我们可以删除不再需要的文件。以下代码展示了如何删除文件。

In [ ]:
import requests
import os

STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
BASE_URL= os.environ['STEPFUN_ENDPOINT']

file_id = "file-KIcn9wB3ZI"
URL = BASE_URL+"/files/"+file_id

headers = {
    "Authorization": f"Bearer {STEP_API_KEY}"
}

response = requests.delete(URL, headers=headers)
responseJson = response.json()
print(responseJson)

## 总结

本教程展示了如何使用阶跃星辰(StepFun)的文档解析API进行文件的上传、状态检查、内容获取、文件列表获取以及文件删除等操作。通过这些API，您可以轻松地将文档处理功能集成到您的应用程序中。

请注意，在实际使用中，您需要替换示例中的文件ID和路径为您自己的文件信息。